<a href="https://colab.research.google.com/github/Amplil/nlp-test/blob/master/transformer_kyoto_ja_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

transformerによる日本語から英語への機械翻訳

コーパスは[http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz](http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz)を使う。

In [ ]:
ls

sample_data/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/.shortcut-targets-by-id/1liJWbv2EhngAj7pixTjzRpv9B-Bz9zCt/Colab Notebooks


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")
import slacknotice # オリジナルモジュール

## 毎回必要なインストール

In [ ]:
pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.3 MB/s 
     |████████████████████████████████| 73 kB 1.9 MB/s 
     |████████████████████████████████| 16.0 MB 118 kB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 39.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


## 前処理

In [ ]:
f = open('./data/kftt-data-1.0/data/tok/kyoto-train.en', 'r')
f.read(500)

'Known as Sesshu ( 1420 - 1506 ) , he was an ink painter and Zen monk active in the Muromachi period in the latter half of the 15th century , and was called a master painter .\nHe revolutionized the Japanese ink painting .\nHe was given the posthumous name " Toyo " or " Sesshu ( 拙宗 ) . "\nBorn in Bicchu Province , he moved to Suo Province after entering SShokoku-ji Temple in Kyoto .\nLater he accompanied a mission to Ming Dynasty China and learned Chinese ink painting .\nHis works were many , includin'

In [ ]:
f = open('./data/kftt-data-1.0/data/tok/kyoto-train.ja', 'r')
f.read(500)

'雪舟 （ せっしゅう 、 1420 年 （ 応永 27 年 ） - 1506 年 （ 永正 3 年 ） ） は 号 で 、 15 世紀 後半 室町 時代 に 活躍 し た 水墨 画 家 ・ 禅僧 で 、 画聖 と も 称え られ る 。\n日本 の 水墨 画 を 一変 さ せ た 。\n諱 は 「 等楊 （ とうよう ） 」 、 もしくは 「 拙宗 （ せっしゅう ） 」 と 号 し た 。\n備中 国 に 生まれ 、 京都 ・ 相国 寺 に 入 っ て から 周防 国 に 移 る 。\nその 後 遣明 使 に 随行 し て 中国 （ 明 ） に 渡 っ て 中国 の 水墨 画 を 学 ん だ 。\n作品 は 数 多 く 、 中国 風 の 山水 画 だけ で な く 人物 画 や 花鳥 画 も よ く し た 。\n大胆 な 構図 と 力強 い 筆線 は 非常 に 個性 的 な 画風 を 作り出 し て い る 。\n現存 する 作品 の うち 6 点 が 国宝 に 指定 さ れ て お り 、 日本 の 画家 の なか で も 別格 の 評価 を 受け て い る と いえ る 。\nこの ため 、'

In [ ]:
f = open('./data/kftt-data-1.0/data/tok/kyoto-dev.en', 'r')
f.read(500)

'Rinzai Zen Buddhism is one of the Chinese five Houses / seven Schools of Zen ( Rinzai , Igyou , Soto , Unmon , Hogan ) and Gigen RINZAI ( ? - 867 ) of Tang was its founder .\nHe was known as " RINZAI of Katu ( meaning to help wake up the sleeping mind ) " and " General RINZAI " , and rose to the top of Chinese Zen with his distinctive style .\nWith its Zen Talks that try to awaken self awareness through study of Koan , it differs from the Silent Zen of the Soto sect , which just does meditation .\n'

In [ ]:
f = open('./data/kftt-data-1.0/data/tok/kyoto-dev.ja', 'r')
f.read(500)

'臨済 宗 （ 臨濟 宗 、 りんざい しゅう ） は 、 中国 禅 五 家 七 宗 （ ご け しち しゅう ） （ 臨済 、 潙仰 宗 、 曹洞 宗 、 雲門 宗 、 法眼 宗 ） の ひと つ で 、 唐 の 臨済 義 玄 （ ？ -867 年 ） を 宗祖 と する 。\n彼 は 『 喝 の 臨済 』 『 臨済 将軍 』 の 異名 で 知 ら れ 、 豪放 な 家風 を 特徴 と し て 中国 禅 興隆 の 頂点 を 極め た 。\n公案 に 参究 する こと に よ り 見性 し よ う と する 看話 禅 （ かんな ぜん ） で 、 ただ 座禅 する 曹洞 宗 の 黙照 禅 と は この 点 が 異な る 。\n中国 に おけ る 臨済 宗\n臨済 宗 は 、 その 名 の 通り 、 会昌 の 廃仏 後 、 唐 末 の 宗祖 臨済 義 玄 に 始ま る 。\n臨済 は 黄檗 希 運 の 弟子 で あ り 、 河北 省 の 地 を 拠点 と し 、 新興 の 藩鎮 勢力 で あ っ た 成徳 府 藩鎮 の 王 常侍 を 支持 基盤 と し て 宗勢 を 伸張 し た が 、 唐 末 五'

In [ ]:
f = open('./data/kftt-data-1.0/data/tok/kyoto-test.en', 'r')
f.read(500)

'Infobox Buddhist\nDogen was a Zen monk in the early Kamakura period .\nThe founder of Soto Zen\nLater in his life he also went by the name Kigen .\nWithin the sect he is referred to by the honorary title Koso .\nPosthumously named Bussho Dento Kokushi , or Joyo-Daishi .\nHe is generally called Dogen Zenji .\nHe is reputed to have been the one that spread the practices of tooth brushing , face washing , table manners and cleaning in Japan .\nAnother story has it that he was the first one to bring Moso-ch'

シリアライズ化

In [ ]:
!onmt_build_vocab -config data/transformer_kyoto_en_ja.yml

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-08-14 06:19:28,013 INFO] Counter vocab from 5000 samples.
[2021-08-14 06:19:28,013 INFO] Build vocab on 5000 transformed examples/corpus.
[2021-08-14 06:19:28,022 INFO] corpus_1's transforms: TransformPipe()
[2021-08-14 06:19:28,024 INFO] Loading ParallelCorpus(data/kftt-data-1.0/data/tok/kyoto-train.en, data/kftt-data-1.0/data/tok/kyoto-train.ja, align=None)...
[2021-08-14 06:19:28,928 INFO] Counters src:13637
[2021-08-14 06:19:28,928 INFO] Counters tgt:12199
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 66, in main
    build_vocab_main(opts)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 53, in build_vocab_main
    save_counter(src_counter, opts.src_vocab)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", lin

In [ ]:
# 恐らく前のバージョンでのコマンドであるため実行できない
#!python OpenNMT-py/preprocess.py -train_src '/content/OpenNMT-py/data/'src-train.txt -train_tgt '/content/OpenNMT-py/data/'tgt-train.txt -valid_src '/content/OpenNMT-py/data/'src-val.txt -valid_tgt '/content/OpenNMT-py/data/'tgt-val.txt -save_data '/content/OpenNMT-py/data/'demo

In [ ]:
# 恐らく前のバージョンでのコマンドであるため実行できない
#!python preprocess.py -train_src train.en.atok -train_tgt train.ja.atok -valid_src dev.en.atok -valid_tgt dev.ja.atok -save_data preprocessed_dataset

## 学習

In [ ]:
f = open('./data/transformer_kyoto_en_ja.yml', 'r')
f.read()

"## Where the samples will be written\nsave_data: data/kftt-data-1.0/run/example\n## Where the vocab(s) will be written\nsrc_vocab: data/kftt-data-1.0/run/example.vocab.src\ntgt_vocab: data/kftt-data-1.0/run/example.vocab.tgt\n# Prevent overwriting existing files in the folder\noverwrite: False\n\n# OpenNMT-py/preprocess.py\ndata:\n    corpus_1:\n        path_src: data/kftt-data-1.0/data/tok/kyoto-train.en\n        path_tgt: data/kftt-data-1.0/data/tok/kyoto-train.ja\n    valid:\n        path_src: data/kftt-data-1.0/data/tok/kyoto-dev.en\n        path_tgt: data/kftt-data-1.0/data/tok/kyoto-dev.ja\n\n# OpenNMT-py/train.py\nsave_model: data/kftt-data-1.0/run/model\nlayers: 6\nrnn_size: 512\nword_vec_size: 512\ntransformer_ff: 2048\nheads: 8\nencoder_type: transformer\ndecoder_type: transformer\nposition_encoding: 'true'\n#train_steps: 200000\ntrain_steps: 15000\nmax_generator_batches: 2\ndropout: 0.1\nbatch_size: 4096\nbatch_type: tokens\nnormalization: tokens\naccum_count: 2\noptim: ada

In [ ]:
!onmt_train -config data/transformer_kyoto_en_ja.yml

[2021-08-14 06:19:29,968 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-08-14 06:19:29,970 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-08-14 06:19:29,970 INFO] Missing transforms field for valid data, set to default: [].
[2021-08-14 06:19:29,971 INFO] Parsed 2 corpora from -data.
[2021-08-14 06:19:29,971 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-08-14 06:19:29,972 INFO] Loading vocab from text file...
[2021-08-14 06:19:29,972 INFO] Loading src vocabulary from data/kftt-data-1.0/run/example.vocab.src
[2021-08-14 06:19:30,378 INFO] Loaded src vocab has 13637 tokens.
[2021-08-14 06:19:30,384 INFO] Loading tgt vocabulary from data/kftt-data-1.0/run/example.vocab.tgt
[2021-08-14 06:19:30,770 INFO] Loaded tgt vocab has 12199 tokens.
[2021-08-14 06:19:30,775 INFO] Building fields with vocab in counters...
[2021-08-14 06:19:30,792 INFO]  * tgt vocab size: 12203.
[2021-08-14 06:19:30,808 IN

In [ ]:
slacknotice.send("train 実行完了")

slack message: train 実行完了


In [ ]:
# 恐らく前のバージョンでのコマンドであるため実行できない
# ->このコマンドをyamlファイルに置き換えたらできるかもしれない->できた
"""
!export CUDA_VISIBLE_DEVICES=0 && \
python train.py -data preprocessed_dataset -save_model save_model_name \
-layers 6 -rnn_size 512 -word_vec_size 512 \
-transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding \
-train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens \
-normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam \
-warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot \
-label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 40000 -world_size 1 -gpu_ranks 0
"""

'\n!export CUDA_VISIBLE_DEVICES=0 && python train.py -data preprocessed_dataset -save_model save_model_name -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 40000 -world_size 1 -gpu_ranks 0 \n'

## 翻訳（検証）

In [ ]:
!onmt_translate -model data/kftt-data-1.0/run/model_step_15000.pt -src data/kftt-data-1.0/data/tok/kyoto-test.en -output data/kftt-data-1.0/out.txt -gpu 0 -verbose

ストリーミング出力は最後の 5000 行に切り捨てられました。
SENT 161: ['Japanese', 'National', 'Railways', 'Steam', 'Locomotive', 'Class', 'D50']
PRED 161: 日本 の <unk> 一覧
PRED SCORE: -5.1505

[2021-08-14 10:38:04,693 INFO] 
SENT 162: ['Manufactured', 'in', '1925', 'by', 'Hitachi', ',', 'Ltd', '.']
PRED 162: 1925 年 （ 大正 14 年 ） 、 常陸 <unk> に よ る <unk> 。
PRED SCORE: -10.5660

[2021-08-14 10:38:04,696 INFO] 
SENT 163: ['Transferred', 'from', 'the', 'Noogata', 'Engine', 'Depot', '.']
PRED 163: <unk> <unk> から の <unk> 。
PRED SCORE: -5.7900

[2021-08-14 10:38:04,697 INFO] 
SENT 164: ['Registration', 'deleted', 'in', '1979', '.']
PRED 164: <unk> 年 に <unk> <unk> 。
PRED SCORE: -6.3943

[2021-08-14 10:38:04,697 INFO] 
SENT 165: ['Japanese', 'National', 'Railways', 'Steam', 'Locomotive', 'Class', 'D51']
PRED 165: 日本 の <unk> 一覧
PRED SCORE: -5.1505

[2021-08-14 10:38:04,697 INFO] 
SENT 166: ['Manufactured', 'in', '1936', 'by', 'Kawasaki', 'Rolling-Stock', 'Mfg', '.', 'Co', '.']
PRED 166: 1936 年 （ 昭和 11 年 ） 、 川崎 <unk> に よ っ て <unk

In [ ]:
!onmt_translate -model data/kftt-data-1.0/run/model_step_15000.pt -src data/en_sample.txt -gpu 0 -verbose -attn_debug -output data/kftt-data-1.0/out_attn.txt

[2021-09-21 12:44:50,574 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2021-09-21 12:44:50,915 INFO] 
SENT 1: ['Tom', 'wants', 'a', 'cat', '.']
PRED 1: <unk> は <unk> を 求め た 。
PRED SCORE: -5.6415

[2021-09-21 12:44:50,915 INFO]                   Tom      wants          a        cat          . 
     <unk>  0.0810000  0.1024836  0.2358939 *0.3071590  0.2734635 
         は  0.0732654  0.2247686  0.1639529  0.1544977 *0.3835154 
     <unk>  0.0123236  0.0434102  0.2558489 *

In [ ]:
onmt_translate

NameError: ignored

In [ ]:
!onmt_translate -model data/kftt-data-1.0/run/model_step_15000.pt -src data/en_sample.txt -gpu 0

[2021-09-21 07:40:05,957 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2021-09-21 07:40:06,071 INFO] PRED AVG SCORE: -0.8059, PRED PPL: 2.2388


# ONMTライブラリを使って翻訳

In [ ]:
import yaml
import torch
import torch.nn as nn
from argparse import Namespace
from collections import defaultdict, Counter

In [ ]:
import onmt
from onmt.inputters.inputter import _load_vocab, _build_fields_vocab, get_fields, IterOnDevice
from onmt.inputters.corpus import ParallelCorpus
from onmt.inputters.dynamic_iterator import DynamicDatasetIter
from onmt.translate import GNMTGlobalScorer, Translator, TranslationBuilder
from onmt.utils.misc import set_random_seed

In [ ]:
# enable logging
from onmt.utils.logging import init_logger, logger
init_logger()

<RootLogger root (INFO)>

In [ ]:
src_vocab_path = "data/kftt-data-1.0/run/example.vocab.src"
tgt_vocab_path = "data/kftt-data-1.0/run/example.vocab.tgt"

In [ ]:
# initialize the frequency counter
counters = defaultdict(Counter)
# load source vocab
_src_vocab, _src_vocab_size = _load_vocab(
    src_vocab_path,
    'src',
    counters)
# load target vocab
_tgt_vocab, _tgt_vocab_size = _load_vocab(
    tgt_vocab_path,
    'tgt',
    counters)

[2021-10-28 09:41:52,299 INFO] Loading src vocabulary from data/kftt-data-1.0/run/example.vocab.src
[2021-10-28 09:41:53,304 INFO] Loaded src vocab has 13637 tokens.
[2021-10-28 09:41:53,314 INFO] Loading tgt vocabulary from data/kftt-data-1.0/run/example.vocab.tgt
[2021-10-28 09:41:53,586 INFO] Loaded tgt vocab has 12199 tokens.


In [ ]:
# initialize fields
src_nfeats, tgt_nfeats = 0, 0 # do not support word features for now
fields = get_fields(
    'text', src_nfeats, tgt_nfeats)

In [ ]:
# build fields vocab
share_vocab = False
vocab_size_multiple = 1
src_vocab_size = 30000
tgt_vocab_size = 30000
src_words_min_frequency = 1
tgt_words_min_frequency = 1
vocab_fields = _build_fields_vocab(
    fields, counters, 'text', share_vocab,
    vocab_size_multiple,
    src_vocab_size, src_words_min_frequency,
    tgt_vocab_size, tgt_words_min_frequency)

[2021-10-28 09:41:53,654 INFO]  * tgt vocab size: 12203.
[2021-10-28 09:41:53,679 INFO]  * src vocab size: 13639.


In [ ]:
src_val = "data/kftt-data-1.0/data/tok/kyoto-test.en"
tgt_val = "data/kftt-data-1.0/data/tok/kyoto-test.ja"

In [ ]:
"""
src_data = {"reader": onmt.inputters.str2reader["text"](), "data": src_val}
tgt_data = {"reader": onmt.inputters.str2reader["text"](), "data": tgt_val}
_readers, _data = onmt.inputters.Dataset.config(
    [('src', src_data), ('tgt', tgt_data)])
"""
src_data = {
    "reader": onmt.inputters.str2reader["text"](),
    "data": src_val,
    "features": {}
}
tgt_data = {
    "reader": onmt.inputters.str2reader["text"](),
    "data": tgt_val,
    "features": {}
}
_readers, _data = onmt.inputters.Dataset.config(
    [("src", src_data), ("tgt", tgt_data)]
)

In [ ]:
dataset = onmt.inputters.Dataset(
    vocab_fields, readers=_readers, data=_data,
    sort_key=onmt.inputters.str2sortkey["text"])

In [ ]:
data_iter = onmt.inputters.OrderedIterator(
            dataset=dataset,
            device="cuda",
            batch_size=10,
            train=False,
            sort=False,
            sort_within_batch=True,
            shuffle=False
        )

In [ ]:
src_text_field = vocab_fields["src"].base_field
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token]

tgt_text_field = vocab_fields['tgt'].base_field
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token]

In [ ]:
emb_size = 100
rnn_size = 500
# Specify the core model.

encoder_embeddings = onmt.modules.Embeddings(emb_size, len(src_vocab),
                                             word_padding_idx=src_padding)

encoder = onmt.encoders.RNNEncoder(hidden_size=rnn_size, num_layers=1,
                                   rnn_type="LSTM", bidirectional=True,
                                   embeddings=encoder_embeddings)

decoder_embeddings = onmt.modules.Embeddings(emb_size, len(tgt_vocab),
                                             word_padding_idx=tgt_padding)
decoder = onmt.decoders.decoder.InputFeedRNNDecoder(
    hidden_size=rnn_size, num_layers=1, bidirectional_encoder=True, 
    rnn_type="LSTM", embeddings=decoder_embeddings)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = onmt.models.model.NMTModel(encoder, decoder)
model.to(device)

# Specify the tgt word generator and loss computation module
model.generator = nn.Sequential(
    nn.Linear(rnn_size, len(tgt_vocab)),
    nn.LogSoftmax(dim=-1)).to(device)

loss = onmt.utils.loss.NMTLossCompute(
    criterion=nn.NLLLoss(ignore_index=tgt_padding, reduction="sum"),
    generator=model.generator)

In [ ]:
src_reader = onmt.inputters.str2reader["text"]
tgt_reader = onmt.inputters.str2reader["text"]
scorer = GNMTGlobalScorer(alpha=0.7, 
                          beta=0., 
                          length_penalty="avg", 
                          coverage_penalty="none")
gpu = 0 if torch.cuda.is_available() else -1
translator = Translator(model=model, 
                        fields=vocab_fields, 
                        src_reader=src_reader, 
                        tgt_reader=tgt_reader, 
                        global_scorer=scorer,
                        gpu=gpu)
builder = onmt.translate.TranslationBuilder(data=dataset, 
                                            fields=vocab_fields)

In [ ]:
for batch in data_iter:
    # アテンションデバッグをON
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=[src_vocab],
        attn_debug=True)
    translations = builder.from_batch(trans_batch)
    for trans in translations:
        print(trans.log(0))
    break


SENT 0: ['Infobox', 'Buddhist']
PRED 0: 下旬 疾病 疾病 疾病 疾病 疾病 疾病 草創 草創 草創 草創 草創 草創 草創 草創 神武 まかえ 生理 生理 るすしき 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 るすしき 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗 拙宗
PRED SCORE: -9.1661


SENT 0: ['Dogen', 'was', 'a', 'Zen', 'monk', 'in', 'the', 'early', 'Kamakura', 'period', '.']
PRED 0: 中堂 にしん にしん にしん にしん 公頼 良薬 良薬 良薬 帯解 くりかえ 個人 にしん にしん にしん にしん 公頼 良薬 良薬 良薬 帯解 くりかえ 個人 にしん にしん にしん にしん 公頼 良薬 良薬 良薬 帯解 壮大 wRaksasa wRaksasa wRaksasa 宋朝 居士 居士 理趣 降魔 出来 降魔 出来 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 降魔 出来 出来 タサダニ
PRED SCORE: -9.1888


SENT 0: ['The', 'founder', 'of', 'Soto', 'Zen']
PRED 0: 毒蛇 おうしん おうしん おうしん 圓 おうしん おうしん ししんごん 圓 おうしん おうしん おうしん ししんごん 圓 おうしん おうしん おうしん ししんごん 圓 おうしん おうしん おうしん ししんごん 圓 おうしん おうしん